General imports

In [1]:
import numpy as np
import sys
sys.path.append('../likelihoods/')
from gaussLike import gaussLike

# Testing $\verb|gaussLike.py|$

In [2]:
# a mock set of data, in this case D i.i.d.'s
# drawn from a unit normal distribution
D = 20
dat = np.random.normal(size=D)
cov = np.diag(np.ones(D))

### A nonzero number of templates

In [3]:
# some ridiculous theory prediction
# has two templates (res2 and res3)
def thy(x,y,z):
    res1 = np.ones(D) * x*y*z
    res2 = np.ones(D) * x
    res3 = np.ones(D) * z
    return np.array([res1,res2,res3]).T

# priors on the two templates
tmp_priors = np.array([[0,1],[1,3]])

like = gaussLike(dat, cov, thy, tmp_priors)

# some basic checks
print('Number of data points',like.D)
print('Number of templates',like.T)
# sanity check the template prior
print("These should be equal",like.templatePrior([0,1]), 1/(1**2 * 3**2 * (2*np.pi)**2)**0.5)

Number of data points 20
Number of templates 2
These should be equal 0.05305164769729845 0.05305164769729845


In [4]:
thy_args = {'x':2,'y':6,'z':0.1}
# this should throw an error
like.rawLogLike(thy_args)

RuntimeError: tmp_prm not specified, but the number of templates is > 0

In [ ]:
# while this shouldn't
like.rawLogLike(thy_args, tmp_prm=[0,0])

In [ ]:
print('Analyically marginalized likelihood',like.margLogLike(thy_args))

# now the brute force way
A = np.linspace(-20,20,200) ; dA = A[1] - A[0]
B = np.linspace(-20,20,200) ; dB = B[1] - B[0]
brute = 0.
for i in range(len(A)):
    for j in range(len(B)):
        brute += np.exp(like.rawLogLike(thy_args, tmp_prm=[A[i],B[j]]))
brute *= dA*dB
print('Brute-force marginalized likelihood',np.log(brute))
print('Relative difference in percent',100*np.abs((like.margLogLike(thy_args)-np.log(brute))/like.margLogLike(thy_args)))

In [ ]:
print('Analytic maximum',like.maxLogLike(thy_args))

# now the brute force way 
from scipy.optimize import minimize
brute = minimize(lambda x: -1*like.rawLogLike(thy_args, tmp_prm=x), x0=np.array([-0.5, -1.2])).fun*-1
print('Brute force max ',brute)

### No templates

In [ ]:
# some ridiculous theory prediction
def thy(x,y,z):
    res1 = np.ones(D) * x*y*z
    return res1

like = gaussLike(dat, cov, thy)

# some basic checks
print('Number of data points',like.D)
print('Number of templates',like.T)
# sanity check the template prior
print("These should be equal",like.templatePrior([0,1]), 1.)

In [ ]:
# this should throw an error
like.anaHelp(thy_args)

In [ ]:
# these should all be equal
print(like.rawLogLike(thy_args))
print(like.margLogLike(thy_args))
print(like.maxLogLike(thy_args))